## Six Step Dance via Langchain
1. Taking into Account Company Industry (Growth / Value)
    1. Growth / Value
    2. Sector outperforming market or not
    3. Company outperforming sector
    4. PE vs Industry
2. Determining Company Financial Stability
    1. Debt To Equity
    2. Current Ratio
    3. Interest Margin Cover
3. Looking at Historical Earning and Growth
    1. Sales Growth
    2. EPS Growth
    3. Return on Invested Capital
    4. Profit Margin
    5. Cashflow Growth
    6. Equity Growth
    7. Return on Equity
4. Understanding Earnings and Sales Expectations
    1. Analyst Estimates
    2. PEG
5. Checking out competition
    1. P/E
    2. P/S
    3.MOAT
6. Estimating Company Values
    1. Insider Buying
    2. Senate TRading
    3. 13f

## Buffett Criteria
### Gross Margin Indicates profitability > 40% GrossProfit/Revenue GROSS PM close to 40 for recent years
### SG&A Margin efficiency in managing overhead costs < 30% SG&A Expenses / Gross Profit
### R&D Margin r&d spending relative to profitability < 30% R&D Expenses / Gross Profit
### Interest Expense Margin reliance on debt and financial health < 15 % Interest Expense / Operating Income
### Income Tax Margin Gauges tax efficiency ~ 20% Income Tax / Pre-Tax Income
### Profit Margin Shows overall profitabilty relative to revenue > 20% Net Income / Revenue
### EPS Growth Indicates growth and consistency in earnings >0 and growing Net Income / Shares Outstanding




### Attempting to write a Small Agent (Top Down)

In [4]:
from openbb import obb
import os
from langchain_openai import ChatOpenAI
import logging
from langchain.agents import tool
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

In [ ]:
@tool
def get_industry_performance() -> list:
    """ Return performance by industry for last week, last month, last quarter, last half year and last year"""
    return obb.equity.compare.groups(group='industry', metric='performance').to_llm()

@tool
def get_strong_buy_for_sector(sector : str) -> list :
    """ Return the strong buy recommendation for a given sector"""
    new_sector = '_'.join(sector.lower().split())
    data = obb.equity.screener(provider='finviz', sector=new_sector, recommendation='buy')
    return data.to_llm()

@tool
def get_strong_buy_for_industry(industry : str) -> list :
    """ Return the strong buy recommendation for a given industry"""
    data = obb.equity.screener(provider='finviz', industry=industry, recommendation='buy')
    return data.to_llm()

@tool
def get_best_stock_performers_for_sector(sector:str) -> list :
    """ Return the best  5 stock performers for last week and last month for a given sector"""
    data = obb.equity.screener(provider='finviz', filters_dict={'Sector' : sector, 'Performance' : 'Week Up', 'Performance 2' : 'Month Up'}, limit=5)
    return data.to_llm()

@tool
def get_best_stock_performers_for_industry(industry:str) -> list :
    """ Return the best  5 stock performers for last week and last month for an industry"""
    data = obb.equity.screener(provider='finviz', filters_dict={'Industry' : industry, 'Performance' : 'Week Up', 'Performance 2' : 'Month Up'}, limit=3)
    return data.to_llm()

@tool
def get_swing_trade_companies(industry:str) -> list:
    """ Return companies candidate for swing trading based on their fundamentals
    """
    try:
        swing_dict = {
                    'InstitutionalOwnership': 'Under 60%',
                    'Current Ratio' :  'Over 1.5',
                    'Debt/Equity'   : 'Over 0.5',
                    'Return on Assets' : 'Over +10%',#
                    'Average Volume' : 'Over 100K',
                    'Price': 'Over $10',#
                    'InstitutionalOwnership': 'Under 60%',
                    'EPS growthpast 5 years' : 'Positive (>0%)',#
                    'EPS growthnext 5 years' : 'Positive (>0%)',
                    'P/E' : 'Under 15',#
                    'P/B': 'Under 2', #
                    }
        data  = obb.equity.screener(provider='finviz', industry='beverages_brewers', 
                        filters_dict=swing_dict
                        )
        return data.to_llm()
    except Exception as e:
        logging.info(f'No data found:{str(e)}')
        return []

@tool
def get_candidate_stocks_to_invest(industry:str) -> list:
    ''' Use fundamentals critera  to select companies in the industry which are worth investing into'''
    desc_filters = {
            'Market Cap.': '+Small (over $300mln)',
            'Average Volume': 'Over 200K',
            #'Relative Volume': 'Over 1',
        }
    fund_filters = {
        'Return on Equity': 'Positive (>0%)',
        'Return on Assets' : 'Over +10%',
        'Return on Investment' : 'Over +10%',
        'InstitutionalOwnership': 'Under 60%',
        'Current Ratio' :  'Over 1.5',
        'Debt/Equity'   : 'Over 0.3',
        #'EPS growthnext 5 years' : 'Positive (>0%)',
    }

    desc_filters.update(fund_filters)

    try:
        data = obb.equity.screener(provider='finviz', industry='semiconductors', 
                    filters_dict=desc_filters
                    )
        return data.to_llm()
    except Exception as e:
        logging.info(f'No data found:{str(e)}')
        return []    

@tool
def get_candidate_stocks_to_invest_relaxxed(industry:str) -> list:
    ''' Use relaxed criterias to find best companies in an industry  which are worth investing into'''
    desc_filters = {
            'Market Cap.': '+Small (over $300mln)',
            'Average Volume': 'Over 200K',
            #'Relative Volume': 'Over 1',
        }
    fund_filters = {
        #'Return on Equity': 'Positive (>0%)',
        #'Return on Assets' : 'Over +10%',
        #'Return on Investment' : 'Over +10%',
        'InstitutionalOwnership': 'Under 60%',
        'Current Ratio' :  'Over 1.5',
        'Debt/Equity'   : 'Over 0.3',
        #'EPS growthnext 5 years' : 'Positive (>0%)',
    }

    desc_filters.update(fund_filters)

    try:
        data = obb.equity.screener(provider='finviz', industry='semiconductors', 
                    filters_dict=desc_filters
                    )
        return data.to_llm()
    except Exception as e:
        logging.info(f'No data found:{str(e)}')
        return []    


@tool
def get_valuation_for_sectors(input:str) -> list:
    """ Return valuation metrics for the sector provided as input"""
    data = obb.equity.compare.groups(group='sector', metric='valuation', provider='finviz').to_df()
    
    filtered =  data[data.name == input]
    return filtered.to_json(
            orient="records",
            date_format="iso",
            date_unit="s",
        )

@tool
def get_valuation_for_industries(input:str) -> list:
    """ Return valuation meetrics for the industry provided as input"""
    data =  obb.equity.compare.groups(group='industry', metric='valuation', provider='finviz').to_df()
    filtered =  data[data.name == input]
    return filtered.to_json(
            orient="records",
            date_format="iso",
            date_unit="s",
        )

@tool
def get_valuation_for_company(ticker:str) -> list:
    """ Return Valuation metrics for a company"""
    # requires obb login
    obb.account.login(pat=os.environ['PAT_KEY'])
    return obb.equity.fundamental.ratios(symbol=ticker, provider='fmp', limit=1).to_llm()

@tool
def get_income_statement(ticker:str) -> list:
    """ Return the last 3  annual income statement for  a given ticker"""
    fields_to_extract = ['total_revenue', 'gross_profit', 'cost_of_revenue', 'net_income' ]
    data = obb.equity.fundamental.income(symbol=ticker, limit=3, provider='yfinance').to_df()
    
    filtered = data[fields_to_extract]

    return filtered.to_json(
            orient="records",
            date_format="iso",
            date_unit="s",
        )

@tool
def get_company_metrics(ticker:str) -> list:
    """ Fetch metrics for a company. It will return the following metrics:
        - pe_ratio
        - foward_pe	
        - eps
        - price_to_sales	
        - price_to_book	
        - book_value_per_share	
        - price_to_cash	
        - cash_per_share	
        - price_to_free_cash_flow	
        - debt_to_equity	
        - long_term_debt_to_equity	
        - quick_ratio	
        - current_ratio	
        - gross_margin	
        - profit_margin	
        - operating_margin	
        - return_on_assets	
        - return_on_investment	
        - return_on_equity	
        - payout_ratio	
    """
    obb.account.login(pat=os.environ['PAT_KEY'])
    data = obb.equity.fundamental.metrics(symbol=ticker)
    if data:
        return data.to_json(
                orient="records",
                date_format="iso",
                date_unit="s",
            )
    return []

@tool
def get_income_growth_for_company(ticker:str) -> list:
    """ Return income growth for a company for the last 3 years,  returnng fields such as:
    1. growth_revenue
    2. growth_basic_earings_per_share
    3. Return on Invested Capital
    4. growth_gross_profit_margin
    5. Cashflow Growth
    6. Equity Growth
    7. Return on Equity
    """
    obb.account.login(pat=os.environ['PAT_KEY'])
    fields_to_extract = ['growth_revenue', 'growth_basic_earings_per_share', 'growth_gross_profit_margin']
    data = obb.equity.fundamental.income(symbol=ticker, limit=3).to_df()
    
    if data:
        filtered = data[fields_to_extract]

        return filtered.to_json(
                orient="records",
                date_format="iso",
                date_unit="s",
            )
    return []

@tool
def get_cashflow_growth_for_company(ticker:str) -> list:
    """ Return cashflow growth for a company for the last 3 years,  returnng fields such as:
    - growth_free_cash_flow
    - growth_operating_cash_flow 
    """
    obb.account.login(pat=os.environ['PAT_KEY'])
    fields_to_extract = ['growth_free_cash_flow', 'growth_operating_cash_flow']
    data = obb.equity.fundamental.cash_growth(symbol=ticker, limit=3).to_df()
    
    if data:
        filtered = data[fields_to_extract]

        return filtered.to_json(
                orient="records",
                date_format="iso",
                date_unit="s",
            )
    return []
    
@tool
def get_balance_sheet_growth(ticker:str)-> list:
    """ Return balance sheet growth for a company, including
    - growth_common_stock
    - growth_total_shareholders_equity
    """
    obb.account.login(pat=os.environ['PAT_KEY'])
    fields_to_extract = ['growth_total_shareholders_equity', 'growth_total_investments', 'growth_common_stock']
    data = obb.equity.fundamental.balance_growth(symbol=ticker, limit=3).to_df()
    
    if data:
        filtered = data[fields_to_extract]

        return filtered.to_json(
                orient="records",
                date_format="iso",
                date_unit="s",
            )
    return []

@tool
def get_profitability_ratio_for_company(ticker:str) -> list:
    """ Return profitability ratios for a company for last 3 years, including
        - return_on_assets
        - return_on_equity
        - return_on_capital_employed
    
    """
    obb.account.login(pat=os.environ['PAT_KEY'])
    
    profitability_ratios = [
                            'return_on_assets',
                            'return_on_equity',
                            'return_on_capital_employed',
                         ]
    data = obb.equity.fundamental.ratios(symbol=ticker, period="annual", limit=3)

    data = obb.equity.fundamental.balance_growth(symbol=ticker, limit=3).to_df()
    
    if data:
        filtered = data[profitability_ratios]

        return filtered.to_json(
                orient="records",
                date_format="iso",
                date_unit="s",
            )
    return []


@tool
def get_buffet_six(industry:str) -> list:
    """ Return companies for an industry which follow warren buffet six criteria for profitability"""
    new_industry = '_'.join(industry.lower().split())
    desc_filters = {
                   'InstitutionalOwnership': 'Under 60%',
                   'Current Ratio' :  'Over 1.5',
                   'Debt/Equity'   : 'Over 0.5',
                   'Return on Assets' : 'Over +10%',#
                   'Average Volume' : 'Over 100K',
                   'Price': 'Over $10',#
                   'InstitutionalOwnership': 'Under 60%',
                   'EPS growthpast 5 years' : 'Positive (>0%)',#
                   'P/E' : 'Under 15',#
                   'P/B': 'Under 2', #
                   }
    
    try:
        data = obb.equity.screener(provider='finviz', industry=new_industry, 
                        filters_dict=desc_filters
                        ).to_df()
        
        if data.shape[0] > 0:
            return data.to_json(
                orient="records",
                date_format="iso",
                date_unit="s",
            )
        else:
            return [f'No companies matching buffet six criterias foudn for {industry}']
    except Exception as e:
        logging.info(str(e))


#get_strong_buy_for_sector.invoke('Consumer Cyclical')
#get_valuation_for_sectors.invoke('Utilities')
#get_valuation_for_company.invoke('AAPL')
#get_valuation_for_industries.invoke('Medical Distribution')
#get_best_stock_performers_for_industry('Utilities')
#get_industry_performance.invoke(None)

#get_income_statement.invoke('MO')
#get_buffet_six.invoke('Broadcasting')
#get_swing_trade_companies.invoke('Broadcasting')
get_candidate_stocks_to_invest.invoke('Broadcasting')


No ticker found.


[]

### Chat Memory

In [11]:
from langchain_core.prompts import MessagesPlaceholder
from langchain.memory import ConversationTokenBufferMemory
#from langchain_core.messages import count_tokens_approximately
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser
from langchain_core.output_parsers import StrOutputParser, CommaSeparatedListOutputParser
from langchain.agents import AgentExecutor
from langchain_core.messages import AIMessage, HumanMessage

MEMORY_KEY = "chat_history"
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """ You are very powerful stock financial researcher, but don't know current events. 
                You  will take the user questions and answer using the tools available.
                Once you have the information you  need, you will answer user's questions using the data returned.
                Use the each of your tools only if explicitely required by the query""",
        ),
        MessagesPlaceholder(variable_name=MEMORY_KEY),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [12]:
tools = [get_industry_performance, get_best_stock_performers_for_industry, get_valuation_for_industries, 
         get_buffet_six, get_income_statement, get_swing_trade_companies]
llm_with_tools = llm.bind_tools(tools)

In [13]:
chat_history = []
chat_history.append(HumanMessage(content="Your question here"))
chat_history.append(AIMessage(content="AI response here"))
memory = ConversationTokenBufferMemory(
    llm=llm,  # Required for token counting
    max_token_limit=18000,  # Leave buffer for functions + responses
    memory_key="chat_history",  # Must match your prompt's key
    return_messages=True
)

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
        "chat_history": lambda x: memory.load_memory_variables(x)["chat_history"],
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

### Attempting a Chain of Thought Prompt

In [ ]:
input1 = '''As a stock researcher, you will try to find the best stocks to invest using a top down approach.
1.  Find the industry that has shown a constant postive performance across quarter, month and week; When evaluating performance please place more emphasis on recent week, month and quarter rather than year or semester.
2.  Extract the valuation metrics for this industry.
3. You will find for this industry the companies that 
3.1 Follow buffet six criteria, ordered by p/e ratio in ascending order. 
3.2 If no companies are returned from buffet six criteria, fetch instead all the companies in this industry which are candidates for swing trading based on their fundamentals.
3.3 If no companies are found in previus step, fetch all companies using relaxed criteria
3.3 If you still haven't found any companies, stop here and give a summary of the industry found explaining its performance and its valuation ratio.
If you find some companies, proceed to the next step.
4. If you find any companies fitting previous criteria,  follow this logic:
4.1. if more than 3 companies are returned, pick the top 3 based on p/e ratio descending.
4.2. if you found 3 companies or less in previous step, use the companies you found and for each of them you will fetch their income statement and extract the following fields:total_revenue, gross_profit, net_income.
5. Fifth: for each of the companies found at previous step, calculate total_revenue, gross_profit and  net_income growth across the years.
6. Summarize your finding describing:
6.1 Best performing industry
6.2 Its valuation ratios
6.3 For each of the companbies you have found, create a table to display the ticker, its total_revenue_growth, its gross_profit growth and net_income_growth.

'''
result = agent_executor.invoke({"input": input1, "chat_history": chat_history})
chat_history.extend(
    [
        HumanMessage(content=input1),
        AIMessage(content=result["output"]),
    ]
)
print(result['output'])



> Entering new AgentExecutor chain...

Invoking: `get_industry_performance` with `{}`


[{"name":"Trucking","performance_1d":-0.0935,"performance_1w":-0.0514,"performance_1m":-0.1509,"performance_3m":-0.3201,"performance_6m":-0.2973,"performance_1y":-0.3444,"performance_ytd":-0.2774,"analyst_recommendation":2.3,"volume":18260000,"volume_average":13150000,"volume_relative":1.4},{"name":"Healthcare Plans","performance_1d":-0.0128,"performance_1w":-0.0461,"performance_1m":-0.113,"performance_3m":-0.0993,"performance_6m":-0.1405,"performance_1y":-0.1326,"performance_ytd":-0.03,"analyst_recommendation":1.65,"volume":33690000,"volume_average":42410000,"volume_relative":0.8},{"name":"Gold","performance_1d":-0.0194,"performance_1w":-0.0402,"performance_1m":0.0963,"performance_3m":0.2916,"performance_6m":0.1167,"performance_1y":0.5122,"performance_ytd":0.4474,"analyst_recommendation":1.78,"volume":194550000,"volume_average":249580000,"volume_relative":0.79},{"name":"Insurance Brokers","perfor

In [ ]:
input1 = '''
Find the industry with the best performance across last week, last month and last quarter and once found get its valuation metrics.
Then summarize the results in less than 100 words explaining performance and valuation metrics.
'''
result = agent_executor.invoke({"input": input1, "chat_history": chat_history})
chat_history.extend(
    [
        HumanMessage(content=input1),
        AIMessage(content=result["output"]),
    ]
)
print(result['output'])

In [ ]:
input1 = '''
what was the best performing industry you found earlier on?
'''
result = agent_executor.invoke({"input": input1, "chat_history": chat_history})
chat_history.extend(
    [
        HumanMessage(content=input1),
        AIMessage(content=result["output"]),
    ]
)
print(result['output'])

### Let's try to do another approach, We start with the industry, find the ones that are performing better than the industry and narrow down by criteria

In [ ]:
input1 = '''
First, analyze the available data to find a industry that has consistently shown positive performance across quarterly, monthly, and weekly timeframes. 
Second, once you have identified the industry, extract its relevant valuation metrics (e.g., P/E, P/B, EV/EBITDA). 
Third, extract companies from the selected industry which follow the buffet six criteria, order by .
Fourth, extract the income statement for  the top performer. 
Finally, using income statement data, compute the revenue growth for the past 3 years.'''
result = agent_executor.invoke({"input": input1, "chat_history": chat_history})
print(result['output'])

In [ ]:
input1 = "Using the recently found strong buys for Utilities , find their valuation metrics and compare it with the Utilities sector valuation metrics"
result = agent_executor.invoke({"input": input1, "chat_history": chat_history})
print(result['output'])

In [ ]:
chat_history.extend(
    [
        HumanMessage(content=input1),
        AIMessage(content=result["output"]),
    ]
)
agent_executor.invoke({"input": "Which stock are recommented for this sector best performing sector", "chat_history": chat_history})